In [2]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import wordnet as wnimport
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import wordnet as wn

In [3]:
f = open('glove.6B.100d 2.txt', 'r') 
vocab = []
embeddings = {}
matrix = []
for line in f:
    split_line = line.split()
    word = split_line[0]
    vector = np.asarray([float(i) for i in split_line[1:]])
    embeddings.update({word:vector})
    vocab.append(word)
    matrix.append(vector)

In [4]:
def evaluate(word, relation, k = 4, return_list = False):
    """Takes in a target word, a list of two words expressing a relation,
    an integer denoting the amount of values that should be returned, and whether a list of words should be returned,
    or only the first item. k is set to 4 by default, in the case that the target word and the relation words
    answer the analogy.
    Returns a list of words of length [k - 3, k] that satisfy the analogy in order of decreasing cosine similarity"""
    if [i for i in [word, relation[0], relation[1]] if i not in embeddings]:
        raise ValueError("Word must be in vocabulary")
    result_vector = embeddings[relation[0]] - embeddings[relation[1]] + embeddings[word]
    nearest_indices = k_nearest_vectors(k, matrix, [result_vector])[0]
    closest_words = [vocab[i] for i in nearest_indices if vocab[i] != word and vocab[i] not in relation]
    if return_list:
        return closest_words
    else:
        return closest_words[0]

In [5]:
def k_nearest_vectors(k, mtx, candidate_vector):
    """Takes in an integer value(k), a matrix (2D list) of all the vectors, and the vector (list) we want to compare.
    Returns an array of length k for indices of the most similar word vectors, and the cosine similarities of these vectors """
    cos_similarities = cosine_similarity(mtx, candidate_vector).flatten()
    k_sorted = np.flip(np.argsort(cos_similarities)[-k:], axis = 0)
    cos_sorted = np.flip(np.sort(cos_similarities), axis = 0)[:k]
    return k_sorted, cos_sorted

# Evaluating

In [6]:
evaluate('king', ['woman', 'man'])

'queen'

# Stereotype and Bias 

In [7]:
evaluate('nurse', ['man', 'doctor']) #gender based on occupation

'woman'

Literally says Nurse to Doc as woman is to man

In [8]:
evaluate('criminal', ['white', 'police']) #racial stereotypes

'brown'

In [9]:
evaluate('terrorist', ['christianity', 'lawful']) #religious stereotypes    

'islamic'

In [10]:
# lets compare cosine similarities
def cosine_sim(target,relation):
    
      return({target:[{relation[0]:cosine_similarity([embeddings[target]],[embeddings[relation[0]]])[0].tolist()},{relation[1]:cosine_similarity([embeddings[target]],[embeddings[relation[1]]])[0].tolist()}]})

In [11]:
cosine_sim('engineer', ['man','woman'])# stereotype in profession
          

{'engineer': [{'man': [0.4299850925969518]}, {'woman': [0.3340311091827807]}]}

In [12]:
cosine_sim('engineer', ['african','american'])# stereotype in profession by location


{'engineer': [{'african': [0.22085807332344548]},
  {'american': [0.42412603855330006]}]}

In [13]:
cosine_sim('terrorist', ['christianity', 'islam'])# religion

{'terrorist': [{'christianity': [0.19160212301043505]},
  {'islam': [0.4048855348671819]}]}

In [92]:
cosine_sim('beauty', ['black', 'white']) # in races

{'beauty': [{'black': [0.42221943915798205]}, {'white': [0.3061925933575558]}]}

# Towards the solution

Neutralize bias for non-gender specific words: I'll use an algorithm by Boliukbasi et al., 2016 to perform gender debiasing. Note that some word pairs such as "actor"/"actress" or "grandmother"/"grandfather" should remain gender specific, while other words such as "receptionist" or "technology" should be neutralized, i.e. not be gender-related.

In [14]:
def neutralize(word, g, word_to_vec_map):
    """
    Removes the bias of "word" by projecting it on the space orthogonal to the bias axis.
    This function ensures that gender neutral words are zero in the gender subspace.
    Arguments:
        word -- string indicating the word to debias
        g -- numpy-array of shape (50,), corresponding to the bias axis (such as gender)
        word_to_vec_map -- dictionary mapping words to their corresponding vectors.
    Returns:
        e_debiased -- neutralized word vector representation of the input "word"
    """

    # Select word vector representation of "word"
    e = embeddings[word]

    # Compute e_biascomponent
    e_biascomponent = np.divide(np.dot(e, g), np.linalg.norm(g) ** 2) * g

    # Neutralize e by substracting e_biascomponent from it
    e_debiased = e - e_biascomponent

    return e_debiased


In [15]:
# Gender concept from GloVe vectors
g = embeddings['woman'] - embeddings['man']


In [16]:
cosine_similarity([embeddings['man']], [g])

array([[-0.18769064]])

In [17]:
cosine_similarity([embeddings['woman']], [g])

array([[0.388177]])

- compared to the gender subspace the word 'man' is -ve and 'woman' is positive

In [18]:
print("\nCosine similarity between a given word and the gender concept\n")
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 'receptionist', 
             'technology',  'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']
for w in word_list:
    print (w, cosine_similarity([embeddings[w]], [g]))


Cosine similarity between a given word and the gender concept

lipstick [[0.18037245]]
guns [[-0.09964446]]
science [[-0.02147577]]
arts [[0.01484675]]
literature [[0.08261854]]
warrior [[-0.156342]]
doctor [[0.10942282]]
tree [[-0.0886836]]
receptionist [[0.28068759]]
technology [[-0.14474527]]
fashion [[0.08097437]]
teacher [[0.15233696]]
engineer [[-0.12300012]]
pilot [[-0.04113394]]
computer [[-0.11545715]]
singer [[0.11372643]]


- we see that words like lipstick and singer are closer to women and guns are closer to men

In [19]:
# before neutralizing
word='receptionist'
print("\ncosine similarity between " + word + " and g, before neutralizing: ", cosine_similarity([embeddings["receptionist"]], [g]))
print("\ncosine similarity between " + word + " and man, before neutralizing: ", cosine_similarity([embeddings["receptionist"]], [embeddings["man"]]))
print("\ncosine similarity between " + word + " and woman, before neutralizing: ", cosine_similarity([embeddings["receptionist"]], [embeddings["woman"]]))



cosine similarity between receptionist and g, before neutralizing:  [[0.28068759]]

cosine similarity between receptionist and man, before neutralizing:  [[0.18974931]]

cosine similarity between receptionist and woman, before neutralizing:  [[0.33642033]]


In [20]:
# after neutralizing the distances are equal
n_word=neutralize('receptionist',g,embeddings)
print("\ncosine similarity between " + word + " and g, after neutralizing: ", cosine_similarity([n_word], [g]))
print("\ncosine similarity between " + word + " and g, after neutralizing: ", cosine_similarity([n_word], [embeddings["man"]]))
print("\ncosine similarity between " + word + " and g, after neutralizing: ", cosine_similarity([n_word], [embeddings["woman"]]))



cosine similarity between receptionist and g, after neutralizing:  [[6.9388939e-18]]

cosine similarity between receptionist and g, after neutralizing:  [[0.2525859]]

cosine similarity between receptionist and g, after neutralizing:  [[0.2369911]]


In [41]:
word='engineer'
print("\ncosine similarity between " + word + " and g, before neutralizing: ", cosine_similarity([embeddings["engineer"]], [g]))



cosine similarity between engineer and g, before neutralizing:  [[-0.12300012]]


In [40]:
n_word=neutralize('engineer',g,embeddings)
print("\ncosine similarity between " + word + " and g, after neutralizing: ", cosine_similarity([n_word], [g]))



cosine similarity between receptionist and g, after neutralizing:  [[-3.46944695e-17]]


In [45]:
print("The distances are almost similar now")
print("\ncosine similarity between " + word + " and woman, after neutralizing: ", cosine_similarity([n_word], [embeddings["woman"]]))
print("\ncosine similarity between " + word + " and man, after neutralizing: ", cosine_similarity([n_word], [embeddings["man"]]))


The distances are almost similar now

cosine similarity between engineer and woman, after neutralizing:  [[0.38469807]]

cosine similarity between engineer and man, after neutralizing:  [[0.41001249]]


# Equalization 
- algorithm for gender-specific words: Equalization is applied to pairs of words that you might want to have differ only through the gender property. As a concrete example, suppose that "actress" is closer to "babysit" than "actor." By applying neutralizing to "babysit" we can reduce the gender-stereotype associated with babysitting. But this still does not guarantee that "actor" and "actress" are equidistant from "babysit." The equalization algorithm takes care of this. The key idea behind equalization is to make sure that a particular pair of words are equidistant from the gender concept encoded by the word embeddings.

In [23]:
def equalize(pair, bias_axis, word_to_vec_map):
    """
    Debias gender specific words by following the equalize method
    Arguments:
    pair -- pair of strings of gender specific words to debias, e.g. ("actress", "actor")
    bias_axis -- numpy-array of shape (50,), vector corresponding to the bias axis, e.g. gender
    word_to_vec_map -- dictionary mapping words to their corresponding vectors
    Returns
    e_1 -- word vector corresponding to the first word
    e_2 -- word vector corresponding to the second word
    """

    # Select word vector representation of "word"
    w1, w2 = pair
    e_w1, e_w2 = word_to_vec_map[w1], word_to_vec_map[w2]

    # Compute the mean of e_w1 and e_w2
    mu = (e_w1 + e_w2) / 2.0

    # Compute the projections of mu over the bias axis and the orthogonal axis
    mu_B = np.divide(np.dot(mu, bias_axis), np.linalg.norm(bias_axis) ** 2) * bias_axis
    mu_orth = mu - mu_B

    # Apply the formula
    e_w1B = np.divide(np.dot(e_w1, bias_axis), np.linalg.norm(bias_axis) ** 2) * bias_axis
    e_w2B = np.divide(np.dot(e_w2, bias_axis), np.linalg.norm(bias_axis) ** 2) * bias_axis
    corrected_e_w1B = np.sqrt(np.abs(1 - np.sum(mu_orth ** 2))) * np.divide(e_w1B - mu_B, np.abs(e_w1 - mu_orth - mu_B))
    corrected_e_w2B = np.sqrt(np.abs(1 - np.sum(mu_orth ** 2))) * np.divide(e_w2B - mu_B, np.abs(e_w2 - mu_orth - mu_B))

    # Debias by equalizing e1 and e2 to the sum of their corrected projections
    e1 = corrected_e_w1B + mu_orth
    e2 = corrected_e_w2B + mu_orth

    return e1, e2


In [24]:
print("\ncosine similarities before equalizing:")
print("\ncosine_similarity(word_to_vec_map[\"actor\"], engineer) = ", cosine_similarity([embeddings["actor"]], [embeddings['engineer']]))
print("cosine_similarity(word_to_vec_map[\"actress\"], engineer) = ", cosine_similarity([embeddings["actress"]], [embeddings['engineer']]))
print()


cosine similarities before equalizing:

cosine_similarity(word_to_vec_map["actor"], engineer) =  [[0.4050496]]
cosine_similarity(word_to_vec_map["actress"], engineer) =  [[0.26930187]]



In [25]:
e1, e2 = equalize(("actor", "actress"), g, embeddings)
print("cosine similarities after equalizing:")
print("\ncosine_similarity(e1, engineer) = ", cosine_similarity([e1], [embeddings['engineer']]))
print("cosine_similarity(e2, engineer) = ", cosine_similarity([e2], [embeddings['engineer']]))
print("\n")

cosine similarities after equalizing:

cosine_similarity(e1, engineer) =  [[0.16561745]]
cosine_similarity(e2, engineer) =  [[-0.10535123]]




- Although the vectors are not exactly equidistant but they are almost